## Лабораторная работа No1. Python и Pandas.

### Задание 1. 

In [53]:
import pandas as pd

In [54]:
transactions = pd.read_csv('transactions.csv', sep=';', decimal=',')
merchants = pd.read_csv('merchants.csv', sep=';', decimal=',')

In [55]:
merged_df = pd.merge(transactions, merchants, on='merchantid')

In [ ]:
count_values = df['column_name'].value_counts()

In [64]:
merged_df = merged_df[merged_df['fraudbits'] == 0]
# merged_df.info()

In [65]:
guild = merged_df['guild'].value_counts()
print(guild)

guild
Гильдия Обработчиков Шерсти         23337
Ассоциация Оливковых Торговцев      13689
Гильдия Пекарей                     11507
Гильдия Гончаров                    10780
Гильдия Парфюмеров                   9212
Гильдия Августовских Торговцев       8862
Братство Кузнецов                    6820
Братство Виноделов                   3898
Лига Керамистов                      3311
Объединение Фруктовых Садовников     2540
Гильдия Мукомолов                     905
Лига Стеклодувов                      891
Объединение Рыбаков                   775
Союз Судостроителей                   650
Союз Каменотесов                      619
Братство Поставщиков Специй           462
Союз Скотоводов                       315
Общество Ювелиров                     146
Name: count, dtype: int64


In [ ]:
# Добавление столбца с амнистией
merged_df['amnesty'] = abs(merged_df['cost'] - merged_df['model_cost']) / merged_df['coef']

In [ ]:
# Группировка данных по гильдиям для вычисления соотношения продаж к покупкам и средней амнистии
guild_stats = (
    merged_df.groupby('guild')
    .agg(
        sale_count=('countertype', lambda x: (x == 'sell').sum()),
        buy_count=('countertype', lambda x: (x == 'buy').sum()),
        total_amnesty=('amnesty', 'mean'),
    )
    .reset_index()
)

guild_stats

,guild,sale_count,buy_count,total_amnesty
0,Ассоциация Оливковых Торговцев,1330,12359,682.928338
1,Братство Виноделов,392,3506,1726.452617
2,Братство Кузнецов,659,6161,1835.734533
3,Братство Поставщиков Специй,47,415,555.012089
4,Гильдия Августовских Торговцев,849,8013,940.918641
5,Гильдия Гончаров,1019,9761,677.369341
6,Гильдия Мукомолов,78,827,447.359102
7,Гильдия Обработчиков Шерсти,2262,21075,1353.118922
8,Гильдия Парфюмеров,872,8340,926.507174
9,Гильдия Пекарей,1174,10333,1026.286880
